# Classification on Wine Dataset

## IMPORTANT: make sure to rerun all the code from the beginning to obtain the results for the final version of your notebook, since this is the way we will do it before evaluting your notebook!!!

### Dataset description

We will be working with a dataset on wines from the UCI machine learning repository
(http://archive.ics.uci.edu/ml/datasets/Wine). It contains data for 178 instances. 
The dataset is the results of a chemical analysis of wines grown in the same region
in Italy but derived from three different cultivars. The analysis determined the
quantities of 13 constituents found in each of the three types of wines. 

### The features in the dataset are:

- Alcohol
- Malic acid
- Ash
- Alcalinity of ash
- Magnesium
- Total phenols
- Flavanoids
- Nonflavanoid phenols
- Proanthocyanins
- Color intensity
- Hue
- OD280/OD315 of diluted wines
- Proline



We first import all the packages that are needed

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt


import numpy as np
import scipy as sp
from scipy import stats
from sklearn import datasets
from sklearn import linear_model

# Perceptron
We will implement the perceptron and use it to learn a halfspace with 0-1 loss.

**TO DO** Set the random seed to your ID (matricola).

In [2]:
IDnumber = 1206150
np.random.seed(IDnumber)

Load the dataset from scikit learn and then split in training set and test set (50%-50%) after applying a random permutation to the datset.

In [3]:
# Load the dataset from scikit learn
wine = datasets.load_wine()

m = wine.data.shape[0]
print('m:'+str(m))
permutation = np.random.permutation(m)

X = wine.data[permutation]
Y = wine.target[permutation]
print(X.shape)

m:178
(178, 13)


We are going to classify class "1" vs the other two classes (0 and 2). We are going to relabel the other classes (0 and 2) as "-1" so that we can use it directly with the perceptron.

In [4]:
#let's relabel classes 0 and 2 as -1

for i in range(len(Y)):
    if Y[i] != 1:
        Y[i] = -1

**TO DO** Divide the data into training set and test set (50% of the data each)

In [5]:
#Divide in training and test: make sure that your training set
#contains at least 10 elements from class 1 and at least 10 elements
#from class -1! If it does not, modify the code so to apply more random
#permutations (or the same permutation multiple times) until this happens.

#m_training needs to be the number of samples in the training set
m_training = m//2

#m_test needs to be the number of samples in the test set
m_test = m-m_training

#this code should change the permutation of the dataset if not heterogenous
isHet=False
while(not(isHet)):
    #X_training = instances for training set
    X_training = X[:m_training,:]
    #Y_training = labels for the training set
    Y_training = Y[:m_training]
    #X_test = instances for test set
    X_test = X[m_training:,:]
    #Y_test = labels for the test set
    Y_test = Y[m_training:]
    countOne=0
    countMOne=0
    for y in Y_training:
        if(y==1):
            countOne = countOne+1
        else:
            countMOne= countMOne+1

    if(countOne>=10)and(countMOne>=10):
        isHet=True
    else:
        print('permuted')
        print(Y_training)
        permutation = np.random.permutation(m)
        X = wine.data[permutation]
        Y = wine.target[permutation]
    
print('n. of 1: '+str(countOne))
print('n.of -1 :'+str(countMOne) )
print(Y_training) #to make sure that Y_training contains both 1 and -1

n. of 1: 37
n.of -1 :52
[-1 -1 -1 -1 -1  1  1 -1  1  1  1  1  1  1 -1 -1 -1  1 -1 -1  1  1 -1 -1
 -1 -1  1 -1 -1  1 -1 -1  1 -1 -1  1 -1  1  1 -1 -1 -1 -1 -1  1 -1  1  1
  1 -1 -1 -1 -1  1  1 -1 -1  1  1  1 -1 -1 -1  1 -1  1  1 -1  1 -1 -1 -1
 -1  1  1 -1 -1  1  1 -1 -1  1 -1 -1 -1  1  1 -1 -1]


**TO DO** Now add a 1 in front of each sample so that we can use a vector to describe all the coefficients of the model. You can use the function $hstack$ in $numpy$

In [6]:
#add a 1 to each sample
X_training = np.hstack((np.ones((m_training,1)),X_training))
X_test = np.hstack((np.ones((m_test,1)),X_test))
print('x_training',X_training.shape)
print('x_test',X_test.shape)


x_training (89, 14)
x_test (89, 14)


**TO DO** Now complete the function *perceptron*. Since the perceptron does not terminate if the data is not linearly separable, your implementation should return the desired output (see below) if it reached the termination condition seen in class or if a maximum number of iterations have already been run, where 1 iteration corresponds to 1 update of the perceptron weights. If the perceptron returns because the maximum number of iterations has been reached, you should return an appropriate model. 

The input parameters to pass are:
- $X$: the matrix of input features, one row for each sample
- $Y$: the vector of labels for the input features matrix X
- $max\_num\_iterations$: the maximum number of iterations for running the perceptron

The output values are:
- $best\_w$: the vector with the coefficients of the best model
- $best\_error$: the *fraction* of missclassified samples for the best model

In [7]:
best_w=np.ones((1,len(X_training[0,:])))
print(best_w)
print(X_training[0,:])
print(best_w*X_training[0,:])
print(np.sum(best_w*X_training[0,:]))
if(np.sign(np.sum(best_w.T*X_training[:,0]))!=np.sign(1)):
    print('alo')
    
print(np.sum(best_w.T*X_training[:,0]))
print(np.sign(np.sum(best_w.T*X_training[:,0])))
print(np.sign(1))

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
[1.000e+00 1.293e+01 2.810e+00 2.700e+00 2.100e+01 9.600e+01 1.540e+00
 5.000e-01 5.300e-01 7.500e-01 4.600e+00 7.700e-01 2.310e+00 6.000e+02]
[[1.000e+00 1.293e+01 2.810e+00 2.700e+00 2.100e+01 9.600e+01 1.540e+00
  5.000e-01 5.300e-01 7.500e-01 4.600e+00 7.700e-01 2.310e+00 6.000e+02]]
747.44
1246.0
1.0
1


In [8]:
def perceptron(X,Y,max_num_iterations):
    len_w=len(X[0,:])
    w=np.zeros((1,len_w))
    for t in range(max_num_iterations):
        for i in range(len(X)):
            wx=np.dot(w,X[i,:])
            #print(wx)
            if Y[i]*np.dot(w,X[i,:]) <=0 :
                w = w + Y[i]*X[i,:]
                
    best_w=w
    
    count=0
    for i in range(len(X)):
        if(np.sign(np.sum(best_w*X[i,:]))!=np.sign(Y[i])):
            count=count+1    
    best_error=count/len(X)
    return best_w, best_error

#now run the perceptron for 100 iterations
# w_found, training_error = perceptron(X_training,Y_training, 100)
# print(w_found)
# print("Training error with 100 iterations: "+str(training_error))

#test with custom matrix
# X_prova = np.array([
#     [-2,2],
#     [-2,-3],
#     [2,-1],
#     [1,-4],
#     [2,2],
#     [3,-1]
# ])
# Y_prova = np.array([1,-1,+1,-1,+1,-1])
# print(X_prova)
# print(Y_prova)
# w_found,training_error=perceptron(X_prova,Y_prova,100)
# print('w_found: ',w_found)
# print('training_error: ',training_error)

Now we use the implementation above of the perceptron to learn a model from the training data using 100 iterations and print the error of the best model we have found.

In [9]:
#now run the perceptron for 100 iterations
w_found, training_error = perceptron(X_training,Y_training, 100)
print(w_found)
print("Training error with 100 iterations: "+str(training_error))

[[  199.    1196.66 -1501.56   173.87  2971.7   7999.     241.81   507.78
     76.83   412.47 -1913.1    512.34   792.14 -1410.  ]]
Training error with 100 iterations: 0.19101123595505617


**TO DO** use the best model $w\_found$ to predict the labels for the test dataset and print the fraction of missclassified samples in the test set (that is an estimate of the true loss).

In [10]:
#now use the w_found to make predictions on test dataset

#num_errors = number of errors in the test set
num_errors = 0.

#ADD CODE!
num_errors=0
for i in range(len(X_test)):
    if(np.sign(np.dot(w_found,X_test[i,:]))!=np.sign(Y_test[i])):
        num_errors=num_errors+1    
true_loss_estimate = num_errors/m_test

#NOTE: you can avoid using num_errors if you prefer, as long as true_loss_estimate is correct
print("Estimated true loss with 100 iterations:"+str(true_loss_estimate))

Estimated true loss with 100 iterations:0.29213483146067415


**TO DO**: what relation do you observe between the training error and the (estimated) true loss? Is this what you expected? Explain what you observe and why it does or does not conform to your expectations.

**ANSWER**: 

**TO DO** Copy the code from the last 2 cells above in the cell below and repeat the training with 10000 iterations. 

In [11]:
#now run the perceptron for 10000 iterations here!

#now run the perceptron for 100 iterations
w_found, training_error = perceptron(X_training,Y_training, 10000)

#training_error = error on the training set
print("Training error with 10000 iterations: "+str(training_error))

num_errors=0
for i in range(len(X_test)):
    if(np.sign(np.dot(w_found,X_test[i,:]))!=np.sign(Y_test[i])):
        num_errors=num_errors+1    
        
true_loss_estimate = num_errors/m_test

#NOTE: you can avoid using num_errors if you prefer, as long as true_loss_estimate is correct
print("Estimated true loss with 10000 iterations:"+str(true_loss_estimate))

Training error with 10000 iterations: 0.06741573033707865
Estimated true loss with 10000 iterations:0.0898876404494382


**TO DO** [Answer the following] What changes in the training error and in the test error (in terms of fraction of missclassified samples)? Explain what you observe. [Write the answer in this cell]

**ANSWER**

# Logistic Regression
Now we use logistic regression, as implemented in Scikit-learn, to predict labels. We first do it for 2 labels and then for 3 labels. We will also plot the decision region of logistic regression.

We first load the dataset again.

In [12]:
# Load the dataset from scikit learn
wine = datasets.load_wine()

m = wine.data.shape[0]
permutation = np.random.permutation(m)

X = wine.data[permutation]
Y = wine.target[permutation]

**TO DO** As for the previous part, divide the data into training and test (50%-50%), relabel classes 0 and 2 as -1. Here there is no need to add a 1 at the beginning of each row, since it will be done automatically by the function we will use.

In [15]:
#Divide in training and test: make sure that your training set
#contains at least 10 elements from class 1 and at least 10 elements
#from class -1! If it does not, modify the code so to apply more random
#permutations (or the same permutation multiple times) until this happens.
#IMPORTANT: do not change the random seed.

m_training = m//2
m_test = m-m_training

#let's relabel classes 0 and 2 as -1
for i in range(len(Y)):
    if Y[i] != 1:
        Y[i] = -1

#this code should change the permutation of the dataset if not heterogenous
isHet=False
while(not(isHet)):
    #X_training = instances for training set
    X_training = X[:m_training,:]
    #Y_training = labels for the training set
    Y_training = Y[:m_training]
    #X_test = instances for test set
    X_test = X[m_training:,:]
    #Y_test = labels for the test set
    Y_test = Y[m_training:]
    countOne=0
    countMOne=0
    for y in Y_training:
        if(y==1):
            countOne = countOne+1
        else:
            countMOne= countMOne+1

    if(countOne>=10)and(countMOne>=10):
        isHet=True
    else:
        print('permuted')
        print(Y_training)
        permutation = np.random.permutation(m)
        X = wine.data[permutation]
        Y = wine.target[permutation]
    
print('n. of 1: '+str(countOne))
print('n.of -1 :'+str(countMOne) )
print(Y_training) #to make sure that Y_training contains both 1 and -1


n. of 1: 33
n.of -1 :56
[ 1 -1 -1 -1  1 -1 -1  1  1 -1 -1 -1  1  1 -1  1 -1 -1 -1  1 -1 -1  1  1
 -1  1  1 -1 -1  1 -1  1  1 -1 -1  1 -1  1 -1  1 -1 -1 -1  1 -1  1 -1 -1
 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1  1 -1
  1 -1 -1  1  1  1 -1 -1  1 -1  1  1  1 -1 -1  1 -1]


To define a logistic regression model in Scikit-learn use the instruction

$linear\_model.LogisticRegression(C=1e5)$

($C$ is a parameter related to *regularization*, a technique that
we will see later in the course. Setting it to a high value is almost
as ignoring regularization, so the instruction above corresponds to the
logistic regression you have seen in class.)

To learn the model you need to use the $fit(...)$ instruction and to predict you need to use the $predict(...)$ function. See the Scikit-learn documentation for how to use it.

**TO DO** Define the logistic regression model, then learn the model using the training set and predict on the test set. Then print the fraction of samples missclassified in the training set and in the test set.

In [ ]:
#part on logistic regression for 2 classes
logreg = #COMPLETE

#learn from training set

#ADD CODE!

#predict on training set

#ADD CODE!

#print the error rate = fraction of missclassified samples
print("Error rate on training set: "+str(error_rate_training))

#predict on test set

#ADD CODE!

#print the error rate = fraction of missclassified samples

print("Error rate on test set: "+str(error_rate_test))

Now we do logistic regression for classification with 3 classes.

**TO DO** First: let's load the data once again (with the same permutation from before).

In [ ]:
#part on logistic regression for 3 classes

#Divide in training and test: make sure that your training set
#contains at least 10 elements from each of the 3 classes!
#If it does not, modify the code so to apply more random
#permutations (or the same permutation multiple times) until this happens.
#IMPORTANT: do not change the random seed.
X = wine.data[permutation]
Y = wine.target[permutation]

X_training = #COMPLETE
Y_training = #COMPLETE

X_test = #COMPLETE
Y_test = #COMPLETE

**TO DO** Now perform logistic regression (instructions as before) for 3 classes, learning a model from the training set and predicting on the test set. Print the fraction of missclassified samples on the training set and the fraction of missclassified samples on the test set.

In [ ]:
#part on logistic regression for 3 classes
logreg = #COMPLETE

#learn from training set

#ADD CODE!

#predict on training set

#ADD CODE!

#print the error rate = fraction of missclassified samples
print("Error rate on training set: "+str(error_rate_training))

#predict on test set

#ADD CODE!

#print the error rate = fraction of missclassified samples
print("Error rate on test set: "+str(error_rate_test))

**TO DO** Now pick two features and restrict the dataset to include only two features, whose indices are specified in the $feature$ vector below. Then split into training and test.

In [ ]:
#to make the plot we need to reduce the data to 2D, so we choose two features

features_list = ['Alcohol',
'Malic acid',
'Ash',
'Alcalinity of ash',
'Magnesium',
'Total phenols',
'Flavanoids',
'Nonflavanoid phenols',
'Proanthocyanins',
'Color intensity',
'Hue',
'OD280/OD315 of diluted wines',
'Proline']
labels_list = ['class_0', 'class_1', 'class_2']

index_feature1 = #COMPLETE
index_feature2 = #COMPLETE
features = [index_feature1, index_feature2]

feature_name0 = features_list[features[0]]
feature_name1 = features_list[features[1]]

#X_red is X reduced to include only the 2 features of
#indices index_feature1 and index_feature2
X_red = X[:,features]

X_red_training = #COMPLETE
Y_training = #COMPLETE

X_red_test = #COMPLETE
Y_test = #COMPLETE

Now learn a model using the training data.

In [ ]:
#ADD CODE! (only for learning from training data)

If everything is ok, the code below uses the model in $logreg$ to plot the decision region for the two features chosen above, with colors denoting the predicted value. It also plots the points (with correct labels) in the training set. It makes a similar plot for the test set.

In [ ]:
# Plot the decision boundary. For that, we will assign a color to each
# point in the mesh [x_min, x_max]x[y_min, y_max].
h = .02  # step size in the mesh
x_min, x_max = X_red[:, 0].min() - .5, X_red[:, 0].max() + .5
y_min, y_max = X_red[:, 1].min() - .5, X_red[:, 1].max() + .5
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

Z = logreg.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)

plt.figure(1, figsize=(4, 3))
plt.pcolormesh(xx, yy, Z, cmap=plt.cm.Paired)

# Plot also the training points
plt.scatter(X_red_training[:, 0], X_red_training[:, 1], c=Y_training, edgecolors='k', cmap=plt.cm.Paired)
plt.xlabel(feature_name0)
plt.ylabel(feature_name1)

plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.xticks(())
plt.yticks(())
plt.title('Training set')

plt.show()

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(1, figsize=(4, 3))
plt.pcolormesh(xx, yy, Z, cmap=plt.cm.Paired)

# Plot also the test points 
plt.scatter(X_red_test[:, 0], X_red_test[:, 1], c=Y_test, edgecolors='k', cmap=plt.cm.Paired, marker='s')
plt.xlabel(feature_name0)
plt.ylabel(feature_name1)

plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.xticks(())
plt.yticks(())
plt.title('Test set')

plt.show()